In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# 读取分类后的CSV文件
df = pd.read_csv('nodejs_cve_data_with_types.csv', encoding='ISO-8859-1')

# 提取漏洞名称中的年份
def extract_year(cve_name):
    match = re.search(r'CVE-(\d{4})', cve_name)
    if match:
        return int(match.group(1))  # 提取并返回年份
    return None  # 如果没有匹配到年份，返回 None

# 创建一个新列 'Year'，存储漏洞的发布年份
df['Year'] = df['Name'].apply(extract_year)

# 处理缺失值：将 'NaN' 填充为 0 或删除含有 'NaN' 的行
df['Year'].fillna(0, inplace=True)

# 确保年份列是整数类型
df['Year'] = df['Year'].astype(int)

# 统计每年每种类型的漏洞数量
type_year_counts = df.groupby(['Year', 'Type']).size().unstack(fill_value=0)

# 第一个图表：展示每年不同漏洞类型的数量（堆叠柱状图）
plt.figure(figsize=(10, 6))
type_year_counts.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.xlabel('Year')
plt.ylabel('Number of Vulnerabilities')
plt.title('Vulnerabilities by Year and Type')
plt.xticks(rotation=45, ha='right')  # 设置x轴标签倾斜角度
plt.tight_layout()
plt.show()

# 步骤 1: 检查数据的基本统计信息，找出可能的异常值
print(type_year_counts.describe())

# 步骤 2: 删除年份小于2000的异常数据（根据实际情况调整）
df = df[df['Year'] >= 2000]

# 统计每年发布的漏洞数量（删除异常值后的）
year_counts = df['Year'].value_counts().sort_index()

# 步骤 3: 对漏洞数量进行对数变换，避免极端值的影响
year_counts_log = np.log1p(year_counts)  # 对数变换

# 准备数据：年份（X）和经过对数变换后的漏洞数量（Y）
years = year_counts.index.values.reshape(-1, 1)  # X: 年份
vuln_counts_log = year_counts_log.values  # Y: 对数变换后的漏洞数量

In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# 创建线性回归模型
model = LinearRegression()

# 拟合模型
model.fit(years, vuln_counts_log)

# 预测未来的漏洞数量
future_years = np.array([2026, 2027, 2028, 2029, 2030]).reshape(-1, 1)  # 预测未来5年
predicted_counts_log = model.predict(future_years)

# 将预测值从对数变换还原
predicted_counts = np.expm1(predicted_counts_log)

# 第二个图表：展示实际数据与预测数据的对比
plt.figure(figsize=(10, 6))
plt.plot(years, year_counts, label='Actual Data', color='blue', marker='o')
plt.plot(future_years, predicted_counts, label='Predicted Data', color='red', linestyle='--', marker='x')

# 添加图表标题和标签
plt.xlabel('Year')
plt.ylabel('Number of Vulnerabilities')
plt.title('Vulnerability Trend and Future Predictions')
plt.legend()
plt.xticks(np.append(years, future_years), rotation=45, ha='right')  # 设置x轴标签倾斜角度，调整显示的年份
plt.tight_layout()
plt.show()

# 输出未来的预测结果
for year, count in zip(future_years.flatten(), predicted_counts):
    print(f"Predicted vulnerabilities in {year}: {int(count)}")


NameError: name 'years' is not defined